# Convolutional Neural Networks

### Practical Session

<br/> Prof. Dr. Georgios K. Ouzounis
<br/> email: georgios.ouzounis@go.kauko.lt

## Contents

1. The challenge
2. Data loading
3. Data preprocessing
4. Compile the CNN
5. Make new predictions
6. Improvements

## The challenge

<img src="https://storage.googleapis.com/kaggle-datasets-images/144468/337794/1ad8516fa7ee2c9eadfbdda2b62a7b20/data-original.jpg?t=2019-03-21-04-51-44"/>

Given an image data-set of cats and dogs, train a CNN to differentiate between them and test it out on new images!

## Data loading 

To obtain the data-set for this exercise go to [https://github.com/georgiosouzounis/dogs_and_cats](https://github.com/georgiosouzounis/dogs_and_cats) and follow the instructions.

To obtain the original data-set from [Kaggle.com](kaggle.com), login in to your account and download it from: [https://www.kaggle.com/c/dogs-vs-cats/data](https://www.kaggle.com/c/dogs-vs-cats/data)


We have two datasets, the training and test set. Images in each set differ between them in size, dimensions, color range and semantics (i.e. background scenes)


| training data-set   | test data-set       |
|---------------------|---------------------|
| 4000 images of dogs | 1000 images of dogs |
| 4000 images of cats | 1000 images of cats |

<img src="http://bit.ly/29ltoLY"/>

## Data pre-processing

We need to:

- Normalize the intensities of all images;
- Normalize the size of all images and pad the gaps;
- Create new images to capture unseen ‘views’;
- Set the way images will be ‘fed’ into the network;
- Specify the batch size.

Keras provides a single tool for all that, the [ImageDataGenerator](https://keras.io/preprocessing/image/)  from the Image Preprocessor library. It is primarily used for generating batches of tensor image data with real-time data augmentation. The data will be looped over (in batches). 

In [1]:
#import the keras image data generator library
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

### Image normalization and generation

The project data (images) are separated in two directories, the training and testing images. 

All images need to be rescaled (intensity) to the range of 0 to 1. Given that each of the red, green and blue channels have a highest possible value of 255, re-scaling is done by dividing with 255 (first parameter of the ImageDataGenerator) 

Optionally you may choose to perform contrast correction operations beforehand, as shown on the left.

Next set shear and zoom range values and enable the horizontal flip option.

In [2]:
# configure the training image generator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# configure the test image generator
test_datagen = ImageDataGenerator(rescale = 1./255)


Shear, zoom and horizontal flip generate image distortions that mimic possible object perception from non-available viewing angles.

The image on the left shows an example of the STOP traffic sign distorted in a number of different ways.

[Example of image transformations:](https://chatbotslife.com/german-sign-classification-using-deep-learning-neural-networks-98-8-solution-d05656bf51ad)

<img src="https://miro.medium.com/max/1400/1*_mgTLhXwWGDEgz_2C7dRDg.png"/>

[example of traffice-sign pre-processing](https://campushippo.com/lessons/build-a-tensorflow-traffic-sign-classifier-at-95-accuracy-214727f24)

<img src="https://cdn.filestackcontent.com/xQUoihTkRLA4CBtb3Cll" />

### Import the datasets 

Each image set (training and testing) can be now accessed using a pointer to the respective ImageDataGenerator object of Keras and using the [flow_from_directory()](https://keras.io/preprocessing/image/) function.

<img src="https://miro.medium.com/max/1400/1*IqWwrQJk2-iILjKCAdPQ6w.png" width="500"/>

In [8]:
#Importing the training dataset
training_set = train_datagen.flow_from_directory('dogs_and_cats/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


Found 8000 images belonging to 2 classes.


In [9]:
#Importing the test dataset
test_set = test_datagen.flow_from_directory('dogs_and_cats/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')


Found 2000 images belonging to 2 classes.


In doing so, we need to specify:

- the target image patch size to which each image will be resized to;
- the batch size, i.e. how many images will be thrown to the CNN at each time;
- and the class annotation mode; binary in this case.


## Compiling the CNN


First let us load the necessary libraries from the Keras API:

- Import the [sequential model](https://keras.io/getting-started/sequential-model-guide/);
- Import the [2D convolution layer](https://keras.io/layers/convolutional/);
- Import the [2D max-pooling layer](https://keras.io/layers/pooling/);
- Import the [flatten layer](https://keras.io/layers/core/);  
- Import the [Dense layer](https://keras.io/layers/core/);


In [10]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


### CNN initialization

Create an instance of the sequential model called classifier

<img src="https://www.samyzaf.com/ML/pima/nn6.png"/>

In [11]:
# Initialising the CNN
classifier = Sequential()

### Add a Convolution Layer

Add a convolutional layer using the add method of the sequential model. 

The first argument in the layer type.

We can now customize the:

- Number of filters (neurons) = 32,
- Each Filter size (3x3),
- The image input shape: 64 x 64 pixels x 3 channels (RGB);
- The layer activation function function: ReLU;

In [12]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

Instructions for updating:
Colocations handled automatically by placer.


### Add a Pooling Layer

Lets us add a pooling layer to subsample the input image and capture its properties at half the scale.

We use the same macro as before, **classifier.add()**, specify the layer type to be **MaxPooling2D**, and customize the pooling parameter to **(2,2)**, i.e divide each input dimension by 2.

In [13]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

### Add a Second Convolution Layer

We add a second convolution layer to compute the same filters on the subsampled feature map, i.e. the output of the last pooling layer.

When done we apply a max-pooling once again to reduce the output size.


In [14]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

### Flattening

We compute a layer flattening on the output of the second max-pooling layer. 

This generates stand alone units that can be directed in a conventional ANN.


In [15]:
# Step 3 - Flattening: convert pixels to input nodes
classifier.add(Flatten())

### Full Connection Layers

The ANN complementing the CNN has 1 hidden layer and 1 output. Both are designated as ‘dense layers’. 

The **hidden layer** can be configured with any number of neurons, **128** in this case but that can grow if dropout is to be introduced. It has a **ReLU** activation layer.

The **output layer** has one unit since we want a single label out. If activated the image shows a dog and if not the image shows a cat. To convert the output probability to a class label we use the **sigmoid activation function**.

In [16]:
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

The sigmoid function is used for the two-class logistic regression, whereas the softmax function is used for the multiclass logistic regression (a.k.a. MaxEnt, multinomial logistic regression, softmax Regression, Maximum Entropy Classifier). [Read more on their differences](http://dataaspirant.com/2017/03/07/difference-between-softmax-function-and-sigmoid-function/)

### Compile the CNN

Finally, lets compile the classifier configured with:

- the [Adam optimizer](https://keras.io/optimizers/) for utilizing the **Stochastic Gradient Descent** approach: [article](https://arxiv.org/abs/1412.6980v8) 
- the [binary_crossentropy](https://keras.io/losses/) as a loss function (or objective function, or optimization score function) which is optimal for binary classification problems;
- the [accuracy metric](https://keras.io/metrics/). A metric is a function that is used to judge the performance of your model. A metric function is similar to a loss function, except that the results from evaluating a metric are not used when training the model.

In [17]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', 
                         loss = 'binary_crossentropy', 
                         metrics = ['accuracy'])


## Train the CNN

### Fit the CNN to the Training set

By contrast to the fit() function used in ANNs, we will know use the fit_generator(). For a detailed discussion on the differences between the two read:

[A thing you should know about Keras if you plan to train a deep learning model on a large dataset](https://towardsdatascience.com/keras-a-thing-you-should-know-about-keras-if-you-plan-to-train-a-deep-learning-model-on-a-large-fdd63ce66bd2).

The arguments in order are:

- the training set configured earlier;
- the number of steps in each epoch;
- the number of epochs (re-runs);
- the validation data (validation is an internal process);
- the number of validation steps;


<img src="https://miro.medium.com/max/1400/1*_lQtIO-FvsPcyq3n2HyyCg.png" width="500"/>

In [19]:
# Fitting the ANN to the Training set
classifier.fit_generator(training_set,
                                 steps_per_epoch = 8000,
                                 epochs = 25,
                                 validation_data = test_set,
                                 validation_steps = 2000)


Epoch 1/1
8000/8000 [==============================] - 1442s 180ms/step - loss: 0.3431 - acc: 0.8459 - val_loss: 0.5553 - val_acc: 0.7935


### Fit the CNN to the Training set

Recalling the earlier slide on importing the datasets, we used the flow_from_directoty() function differs from flow as shown on the left diagram.

[steps_per_epoch](https://keras.io/models/sequential/): Total number of steps (batches of samples) to yield from generator before declaring one epoch finished and starting the next epoch. It should typically be equal to the number of unique samples of your dataset divided by the batch size.

In this exercise we set it to be equal to the number of unique samples but this comes at an increase in computational cost.


<img src="https://miro.medium.com/max/1400/1*IqWwrQJk2-iILjKCAdPQ6w.png" width="500"/>

## Making Ndew Predictions

### Is it a cat or a dog?

A new observation (image) is given. Given the model we trained can we classify the new image as one showing a cat or one showing a dog.

<img src="https://cdn.editorchoice.com/wp-content/uploads/2019/06/dogtilt.jpg" width="300"/> <img src="https://boygeniusreport.files.wordpress.com/2017/01/cat.jpg?quality=98&strip=all" width="300"/>

### Load Libraries

We need 
- **NumPy** to store new images for compatibility issues;
- the **image** module of the Keras preprocessing library. It will be used for loading new images and for converting them to NumPy arrays as needed.

In [20]:
# Predicting a single new observation
import numpy as np
from keras.preprocessing import image

### Load and Preprocess New Data

We can now load the new image using the image module and reshape it to the target dimensions.

The image is then converted to a NumPy array and a new dimension is added as required by the Keras classifier. 

In [21]:
# Load the new image
test_image = image.load_img('dogs_and_cats/exercise/what_is_it1.jpg', target_size = (64, 64))


In [22]:
# convert it to an array
test_image = image.img_to_array(test_image)

In [23]:
# add an extra dimension for compatibility
test_image = np.expand_dims(test_image, axis = 0) 

### Run the Classifier

In [24]:
# run the classifier on the new image
result = classifier.predict(test_image)

In [25]:
# get the class indices from the training set
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [26]:
# if the classifier result is 1 the prediction is a dog, or a # cat otherwise
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'


In [27]:
prediction

'dog'

### Love the challenges

|example | example |
|---|---|
|<img src="https://pm1.narvii.com/6568/43456984beeff7563e2865947ff646cc2b200b77_hq.jpg" width="300"/> | <img src="https://fc03.deviantart.net/fs22/f/2007/347/a/e/Catdog_by_Sannebe.jpg" width="300"/>|
|<img src="http://photoshopcontest.com/images/fullsize/sgsaavsxlq4bvgkjcpaaa9jsykqkih08ang1.jpg" width="300"/> | <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQCTzNzCPox2GdcpQCcsvZ3xFTWYX05DVmPrkxUZEDmdPyuoWPl" width="300"/>|
|<img src="https://i.ytimg.com/vi/bxKoHe7m59E/maxresdefault.jpg" width="300"/> | <img src="https://i.chzbgr.com/full/9091640832/h68B68331/" width="300"/>|


## Improvements

To understand the performance of your model’s architecture it is sufficient to run it over the validation set. Subsequently k-Fold Cross Validation is not needed.

Classical approaches for model improvement include adjusting the:
- optimizer,
- number of epochs,
- loss function,
- dropout,
- metrics, etc.

You may wish however to adjust the network architecture by including more convolution and dense layers.
